In [ ]:

%matplotlib inline
import seaborn as sns

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import glob

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--remote-debugging-port=9222")



In [ ]:
data = pd.read_csv('genome.txt', sep='\t', na_values='---', dtype={'rsid':'str', 'chromosome':'object', 'position':'int', 'genotype':'str'}, comment='#', names=["rsid", "chromosome", "position", "genotype"])

data.head(50)
data.isna().any()

In [ ]:
data = pd.read_csv('genome.txt', sep='\t', na_values='---', dtype={'rsid':'str', 'chromosome':'object', 'position':'int', 'genotype':'str'}, comment='#', names=["rsid", "chromosome", "position", "genotype"])

data['chromosome'].unique()
data['chromosome'] = data['chromosome'].apply(lambda x:
    re.sub(r'X', r'23', x))
data['chromosome'] = data['chromosome'].apply(lambda x:
    re.sub(r'MT', r'24', x))

    # Figure out what to do with your Y
data = data[data['chromosome'] != 'Y']
data['chromosome'] = data['chromosome'].apply(lambda x: 
	int(x))

    # showing
# data['isDigit'] = data['chromosome'].str.isdigit()
# data = data.loc[(data['isDigit'] == False) & (data['chromosome'] == 'Y')  ]
# data['isDigit'] = data['chromosome'].str.isdigit()
# data = data[data['isDigit'] == False]
# data
	


In [ ]:
chromosome_dict = {1:'1', 2:'2', 3:'3', 4:'4', 5:'5', 
				   6:'6', 7:'7', 8:'8', 9:'9', 10:'10', 
				   11:'11', 12:'12', 13:'13', 14:'14', 
				   15:'15', 16:'16', 17:'17', 18:'18', 
				   19:'19', 20:'20', 21:'21', 22:'22', 
				   23:'X', 24:'MT'}

print(chromosome_dict)
display(data.info())

In [ ]:
#remove whitespace from column name
data.rename({' rsid': 'rsid'}, axis='columns', inplace=True)

#how many snps are there per chromosome?

#solve manually with a for loop
x = []
y = []
for k in chromosome_dict:
    x.append(k)
    y.append(len(data[data.chromosome ==k]))
rsid_per_chromosome = dict(zip(x,y))
rsid_per_chromosome

#or solve with pandas
rsid_per_chromosome_series = data.groupby('chromosome')['rsid'].count()
rsid_per_chromosome_series.columns = ['chromosome','count']
rsid_per_chromosome_series.plot.barh(figsize=(16,9), fontsize=15)
plt.show()

In [ ]:
all_files = glob.glob("./csv from snpedia/*.csv")

snp_df = pd.concat((pd.read_csv(f) for f in all_files))
snp_df

In [ ]:
snp_df['genotype'] = snp_df['Unnamed: 0'].apply(lambda x: 
	re.sub(r'.*([AGCT]);([AGCT])\)', r'\1\2', x))
new_cols = ['rsid', 'magnitude', 'repute', 
'summary', 'genotype']
snp_df.columns = new_cols

snp_df['rsid'] = snp_df['rsid'].map(lambda x : x.lower())
snp_df['rsid'] = snp_df['rsid'].map(lambda x : 
	re.sub(r'([a-z]{1,}[\d]+)\([agct];[agct]\)', 
	r'\1', x))

In [ ]:
null_repute = snp_df[snp_df['repute'].isnull()]
null_summaries = snp_df[snp_df['summary'].isnull()]
null_repute_and_summaries = pd.concat([null_repute,null_summaries]).drop_duplicates().reset_index(drop=True)
display(null_repute_and_summaries)
snp_df['repute'].fillna(value='Neutral', inplace=True)
snp_df['summary'].fillna(value='None', inplace=True)
snp_df.isna().any()


In [30]:
new_df = snp_df.merge(data, how='inner', on=['rsid', 'genotype'], suffixes=('_SNPedia', '_myDNA'))
bad = new_df[new_df['repute'] == 'Bad'].sort_values('magnitude', ascending = False)
neutral = new_df[new_df['repute'] == 'Neutral'].sort_values('magnitude', ascending = False)
good = new_df[new_df['repute'] == 'Good'].sort_values('magnitude', ascending = False)
interesting = new_df[new_df['magnitude'] > 4].sort_values('magnitude', ascending = False)
good 

,rsid,magnitude,repute,summary,genotype,chromosome,position
4,rs4680,2.5,Good,(worrier) advantage in memory and attention tasks,AA,22,19951271
211,rs601338,2.5,Good,resistance to Norovirus infection,AA,19,49206674
5,rs4680,2.5,Good,(worrier) advantage in memory and attention tasks,AA,22,19951271
188,rs9264942,2.2,Good,90% reduction in HIV viral load,CC,6,31274380
561,rs1799990,2.1,Good,Resistance to vCJD (PrP 129 Met/Val heterozygo...,AG,20,4680251
...,...,...,...,...,...,...,...
176,rs1805005,0.0,Good,common in clinvar,GG,16,89985844
175,rs1805007,0.0,Good,normal risk,CC,16,89986117
174,rs2228479,0.0,Good,common in clinvar,GG,16,89985940
173,rs5907,0.0,Good,common in clinvar,GG,22,21134223


In [37]:

# Get the base URL from SNPedia
import time
base_url = 'https://www.snpedia.com/index.php/'

genes = [bad, neutral, interesting]
urlDictionary = {}
for table in genes:
    gene_urls = [base_url + rsid for rsid in table['rsid']]
    print(gene_urls)
    title = f'{table=}'.split('=')[0]
    urlDictionary[title] = gene_urls

print(urlDictionary)
# Initialize Selenium
browser = webdriver.Chrome(chrome_options=options)
# Write a function to visit the SNPedia URLs, click through to PubMed,
# and retrieve the info on the articles for each gene


def scrape_abstracts(urls):

    rsid_list = []
    all_article_title = []
    all_article_citation = []
    all_article_authors = []
    # all_article_abstract = []
    all_article_links = []
    for url in urls:
        link_urls = []
        browser.get(url)  # load url
        rsid = browser.find_element_by_css_selector('.firstHeading').text
        links_elements = browser.find_elements_by_partial_link_text('PMID')

        # get the URLs to the PubMed pages
        for link in links_elements:
            tempUrl = link.get_attribute('href')
            link_urls.append(tempUrl.split("?")[0])
        print(len(link_urls))
        # follow each link element to PubMed
        for element in link_urls:
            browser.get(element)
            time.sleep(2)
            citation_title = browser.find_element(By.XPATH, "//meta[@name='citation_title']")
            article_title = citation_title.get_attribute('content')
            citation_authors = browser.find_element(By.XPATH, "//meta[@name='citation_authors']")
            article_authors = citation_authors.get_attribute('content')
            citation_journal_title = browser.find_element(By.XPATH, "//meta[@name='citation_journal_title']")
            article_citation = citation_journal_title.get_attribute('content')
            # abs = browser.find_element_by_class_name('abstract')

            rsid_list.append(rsid)
            all_article_title.append(article_title)
            all_article_citation.append(article_citation)
            all_article_authors.append(article_authors)
            # all_article_abstract.append(abs)
            all_article_links.append(element)

        # store the information
    df = pd.DataFrame()
    df['rsid'] = rsid_list
    df['article_title'] = all_article_title
    df['article_citation'] = all_article_citation
    df['article_authors'] = all_article_authors
    # df['article_abstract'] = all_article_abstract
    df['link'] = all_article_links

    df = df.drop_duplicates()

    df.index = range(len(df.index))
    return df
writer = pd.ExcelWriter('./results/interesting_genes.xlsx', engine='xlsxwriter')

for key in urlDictionary:
    abstracts_df = scrape_abstracts(urlDictionary[key])
    abstracts_df.to_excel(writer, sheet_name=key)
writer.save()




           rsid  magnitude repute  \
784  rs62514958        5.9    Bad   
171  rs28930069        5.0    Bad   
104  rs28936383        5.0    Bad   
105  rs28936383        5.0    Bad   
203   rs1800462        3.5    Bad   
..          ...        ...    ...   
37    rs1333040        0.0    Bad   
282   rs2308327        0.0    Bad   
489   rs8177374        0.0    Bad   
321  rs10761659        0.0    Bad   
272    rs440446        0.0    Bad   

                                               summary genotype  chromosome  \
784  Non-phenylketonuria hyperphenylalaninemia geno...       GG          12   
171                Hypokalemic periodic paralysis risk       GG           1   
104  Limb-girdle muscular dystrophy-dystroglycanopathy       GG           4   
105  Limb-girdle muscular dystrophy-dystroglycanopathy       GG           4   
203             incapable of detoxifying certain drugs       CC           6   
..                                                 ...      ...         ...   
37

/tmp/ipykernel_1401/4239684641.py:15: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(chrome_options=options)
/tmp/ipykernel_1401/4239684641.py:31: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  rsid = browser.find_element_by_css_selector('.firstHeading').text
/tmp/ipykernel_1401/4239684641.py:32: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  links_elements = browser.find_elements_by_partial_link_text('PMID')


0
0
0
5
